In [7]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [6]:
meteorites = pd.read_csv("./data/meteorite-landings.csv", delimiter=",")
meteorites = meteorites.rename(columns={'name': 'Name', 'id': 'ID', 'nametype': 'Type', 'recclass': 'Class', 'mass': 'Mass', 'fall': 'Fell', 'year': 'Year', 'reclat': 'Latitude', 'reclong': 'Longitude'})

filtered_coords = (meteorites["Longitude"] >= -180) & (meteorites["Longitude"] <= 180) & ((meteorites["Longitude"] != 0) | (meteorites["Latitude"] != 0))
filtered_years = (meteorites["Year"] >= 860) & (meteorites["Year"] <= 2016)
filtered_mass = ~pd.isna(meteorites["Mass"])
filtered_type = ((meteorites["Type"] == "Valid"))
filtered_meteorites = meteorites[filtered_coords & filtered_years & filtered_type & filtered_mass]
filtered_meteorites.drop(["GeoLocation", "Type"], axis=1, inplace=True)
filtered_meteorites["Mass"] = filtered_meteorites["Mass"].div(1000) # convert gram to kilogram
filtered_meteorites.to_csv("./data/cleansed-data.csv")
filtered_meteorites = filtered_meteorites.sort_values(by="Year", ascending=True)
filtered_meteorites

KeyError: 'Longitude'

In [ ]:
cleansed_meteorites = pd.read_csv("./data/cleansed-data.csv", delimiter=",")
img = Image.open("images/map.jpg")
img.thumbnail((360, 200), Image.ANTIALIAS)
ax = plt.subplot()
ax.scatter(cleansed_meteorites["Longitude"], cleansed_meteorites["Latitude"], s=4)
# plt.figure(figsize=(20,15))
ax.imshow(img)
plt.show()
# plt.savefig("./images/testplot.png", format="png")

In [ ]:
# Meteorite landings across the world

fig = px.scatter_geo(filtered_meteorites,lat='Latitude',lon='Longitude',hover_name="Name",color="Mass",projection="natural earth")
fig.update_layout(title = 'Meteorite Landings Across the World', title_x=.5)
fig.show()

In [ ]:
# Figure of meteorite landings in the US

fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = filtered_meteorites['Longitude'],
        lat = filtered_meteorites['Latitude'],
        text = "Mass " + filtered_meteorites['Mass'].astype(str),
        mode = 'markers',
        marker = dict(
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = filtered_meteorites['Mass'],
            cmax = filtered_meteorites['Mass'].max(),
            colorbar_title="Mass of Meteorites"
        )))
fig.update_layout(
        title = 'Meteorites in the US',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True,
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
fig.show()

In [ ]:
# Calculates the the average meteorite mass per year

def GetAverageMeteoriteMass(year, number):
    sum = 0
    for i in range(len(filtered_meteorites)):
        if filtered_meteorites.iloc[i].Year == year:
            sum += filtered_meteorites.iloc[i].Mass
    return sum / number

filter_old = filtered_meteorites['Year'] > 1766
filtered_meteorites_per_year = filtered_meteorites[filter_old]
filtered_meteorites_per_year = filtered_meteorites_per_year['Year'].value_counts()
average_meteorite_mass_per_year = []
for i in range(len(filtered_meteorites_per_year.index)):
    year = filtered_meteorites_per_year.index[i]
    number = filtered_meteorites_per_year.values[i]
    print(GetAverageMeteoriteMass(year, number))
    average_meteorite_mass_per_year.append(GetAverageMeteoriteMass(year, number))
print(average_meteorite_mass_per_year)

In [ ]:
# Figure of the # of meteorites per year and their average mass

fig = px.scatter(filtered_meteorites_per_year,title="Number of Meteorites Per Year",
color=average_meteorite_mass_per_year,
labels={
    "index": "Year",
    "value": "Number of Meteorites",
    "color": "Average Mass"
}
)
fig.update_layout(showlegend=False)
fig.show()  